In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#데이터 전처리

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, Reshape, SeparableConv2D, Conv2D, BatchNormalization, Multiply, Layer,Attention, LayerNormalization, Add
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [ ]:
# 데이터 경로
train_data_dir = '/content/drive/MyDrive/Data/img/train'
validation_data_dir = '/content/drive/MyDrive/Data/img/val'
test_data_dir = '/content/drive/MyDrive/Data/img/test'

# 데이터 증강
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지 불러오기
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5853 images belonging to 4 classes.
Found 1168 images belonging to 4 classes.
Found 1137 images belonging to 4 classes.


#MobileNetV1 기반 모델링(실행환경 T4)

In [ ]:
# Base model 정의
def create_base_model(input_shape):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    return base_model

# Base model 정의
def create_base_model(input_shape):
    # Load the base MobileNet model with pre-trained ImageNet weights
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    # Get the output from the fourth last layer
    layer_index = -4
    output_layer = base_model.layers[layer_index].output

    # Create the new model
    new_model = Model(inputs=base_model.input, outputs=output_layer)

    return new_model

In [ ]:
# Base model 정의
def create_base_model(input_shape):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    return base_model


# 패치 추출 레이어 정의
def patch_extraction_layer():
    return tf.keras.Sequential([
        SeparableConv2D(256, kernel_size=4, strides=4, padding='same', activation='relu'),
        SeparableConv2D(256, kernel_size=2, strides=2, padding='valid', activation='relu'),
        Conv2D(256, kernel_size=1, strides=1, padding='valid', activation='relu')
    ], name='patch_extraction')

# Pre-classification 레이어 정의
def create_pre_classification_layer():
    return tf.keras.Sequential([
        Dense(32, activation='relu'),
        BatchNormalization()
    ], name='pre_classification')

In [ ]:
def create_mobilenet_with_cbam_block(input_shape, num_classes):
    base_model = create_base_model(input_shape)
    inputs = Input(shape=input_shape)
    # 기본 모델
    x = base_model(inputs)
    # 패치 추출 레이어
    x = patch_extraction_layer()(x)
    # GlobalAveragePooling2D 및 Dropout
    x = GlobalAveragePooling2D(name='gap')(x)
    x = Dropout(0.2)(x)
    # 사전 분류 레이어
    x = create_pre_classification_layer()(x)
    # Self-Attention 추가
    attn_output = Attention(use_scale=True)([x, x])
    x = Add()([x, attn_output])
    # 출력층
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model


# 모델 생성 및 컴파일
model = create_mobilenet_with_cbam_block(input_shape=(224, 224, 3), num_classes=4)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

17225924/17225924 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)           3228864   ['input_2[0][0]']             
 onal)                                                                                            
                                                                                                  
 patch_extraction (Sequenti  (None, 1, 1, 256)            411392    ['mobilenet_1.00_224[0][0]']  
 al)                                                                                              
                             

In [ ]:
# MobileNet 모델 로드
base_model = tf.keras.applications.MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# 모델 레이어 목록 출력
for i, layer in enumerate(base_model.layers):
    print(f"{i}: {layer.name}")


0: input_5
1: conv1
2: conv1_bn
3: conv1_relu
4: conv_dw_1
5: conv_dw_1_bn
6: conv_dw_1_relu
7: conv_pw_1
8: conv_pw_1_bn
9: conv_pw_1_relu
10: conv_pad_2
11: conv_dw_2
12: conv_dw_2_bn
13: conv_dw_2_relu
14: conv_pw_2
15: conv_pw_2_bn
16: conv_pw_2_relu
17: conv_dw_3
18: conv_dw_3_bn
19: conv_dw_3_relu
20: conv_pw_3
21: conv_pw_3_bn
22: conv_pw_3_relu
23: conv_pad_4
24: conv_dw_4
25: conv_dw_4_bn
26: conv_dw_4_relu
27: conv_pw_4
28: conv_pw_4_bn
29: conv_pw_4_relu
30: conv_dw_5
31: conv_dw_5_bn
32: conv_dw_5_relu
33: conv_pw_5
34: conv_pw_5_bn
35: conv_pw_5_relu
36: conv_pad_6
37: conv_dw_6
38: conv_dw_6_bn
39: conv_dw_6_relu
40: conv_pw_6
41: conv_pw_6_bn
42: conv_pw_6_relu
43: conv_dw_7
44: conv_dw_7_bn
45: conv_dw_7_relu
46: conv_pw_7
47: conv_pw_7_bn
48: conv_pw_7_relu
49: conv_dw_8
50: conv_dw_8_bn
51: conv_dw_8_relu
52: conv_pw_8
53: conv_pw_8_bn
54: conv_pw_8_relu
55: conv_dw_9
56: conv_dw_9_bn
57: conv_dw_9_relu
58: conv_pw_9
59: conv_pw_9_bn
60: conv_pw_9_relu
61: conv_dw_10


In [ ]:
# base_model의 구조를 출력
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

In [ ]:
# 학습률 조정 콜백
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2, min_delta=0.005, min_lr=1e-7)

# EarlyStopping 콜백
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, min_delta=0.005, restore_best_weights=True)

# 클래스 가중치 추가
train_samples_per_class = np.array([1440, 1489, 1477, 1447])
total_train_samples = np.sum(train_samples_per_class)
class_weights = {i: total_train_samples / (len(train_samples_per_class) * count) for i, count in enumerate(train_samples_per_class)}


In [ ]:
# 모델 훈련
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping],
    class_weight=class_weights
)

Epoch 1/100
183/183 [==============================] - 5379s 29s/step - loss: 1.3127 - accuracy: 0.3649 - val_loss: 1.3814 - val_accuracy: 0.2954 - lr: 0.0010
Epoch 2/100
183/183 [==============================] - 427s 2s/step - loss: 1.1961 - accuracy: 0.4686 - val_loss: 1.3394 - val_accuracy: 0.3185 - lr: 0.0010
Epoch 3/100
183/183 [==============================] - 436s 2s/step - loss: 1.1426 - accuracy: 0.5015 - val_loss: 1.2608 - val_accuracy: 0.4101 - lr: 0.0010
Epoch 4/100
183/183 [==============================] - 428s 2s/step - loss: 1.1039 - accuracy: 0.5264 - val_loss: 1.3247 - val_accuracy: 0.4623 - lr: 0.0010
Epoch 5/100
183/183 [==============================] - 426s 2s/step - loss: 1.0789 - accuracy: 0.5407 - val_loss: 1.4942 - val_accuracy: 0.3716 - lr: 0.0010
Epoch 6/100
183/183 [==============================] - 424s 2s/step - loss: 1.0462 - accuracy: 0.5645 - val_loss: 1.3551 - val_accuracy: 0.3973 - lr: 0.0010
Epoch 7/100
183/183 [==============================] - 4

In [ ]:
# 파인튜닝 과정
base_model = model.layers[1]
base_model.trainable = True

# 사전 학습된 모델의 전체 레이어 갯수
num_layers = len(base_model.layers)

# 훈련되지 않고 고정될 레이어 비율
freeze_ratio = 0.5
num_freeze_layers = int(num_layers * freeze_ratio)

# 하위 레이어 고정, 상위 레이어 해제
for layer in base_model.layers[:num_freeze_layers]:
    layer.trainable = False
for layer in base_model.layers[num_freeze_layers:]:
    layer.trainable = True

# 모델 재컴파일
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)           3228864   ['input_2[0][0]']             
 onal)                                                                                            
                                                                                                  
 patch_extraction (Sequenti  (None, None, None, 256)      411392    ['mobilenet_1.00_224[0][0]']  
 al)                                                                                              
                                                                                              

In [ ]:
# 모델 훈련
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping],
    class_weight=class_weights
)

Epoch 1/100
183/183 [==============================] - 443s 2s/step - loss: 0.9411 - accuracy: 0.6173 - val_loss: 1.0495 - val_accuracy: 0.5865 - lr: 1.0000e-04
Epoch 2/100
183/183 [==============================] - 439s 2s/step - loss: 0.7369 - accuracy: 0.7205 - val_loss: 0.9180 - val_accuracy: 0.6250 - lr: 1.0000e-04
Epoch 3/100
183/183 [==============================] - 432s 2s/step - loss: 0.6282 - accuracy: 0.7700 - val_loss: 0.9931 - val_accuracy: 0.6250 - lr: 1.0000e-04
Epoch 4/100
183/183 [==============================] - 429s 2s/step - loss: 0.5402 - accuracy: 0.8056 - val_loss: 0.9094 - val_accuracy: 0.6430 - lr: 1.0000e-04
Epoch 5/100
183/183 [==============================] - 436s 2s/step - loss: 0.4637 - accuracy: 0.8411 - val_loss: 0.9066 - val_accuracy: 0.6618 - lr: 1.0000e-04
Epoch 6/100
183/183 [==============================] - 430s 2s/step - loss: 0.4170 - accuracy: 0.8585 - val_loss: 0.8833 - val_accuracy: 0.6721 - lr: 1.0000e-04
Epoch 7/100
183/183 [=============

In [ ]:
# 검증 데이터 확인
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

37/37 [==============================] - 61s 2s/step - loss: 0.8748 - accuracy: 0.6926
Validation Loss: 0.8747538924217224
Validation Accuracy: 0.6926369667053223


In [ ]:
# 테스트 데이터 전체 검증
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


36/36 [==============================] - 1091s 31s/step - loss: 0.8245 - accuracy: 0.6974
Test Loss: 0.8244679570198059
Test Accuracy: 0.6974494457244873


In [ ]:
# 모델 저장
model.save('/content/drive/MyDrive/MobileNetV1_02-03.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Sub

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# 이미지 전처리 함수
def load_and_prepare_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.0  # 정규화
    return img_tensor

# 감정을 예측하는 함수
def predict_emotion(model, img_path):
    print("이미지 로딩 중...")
    test_image = load_and_prepare_image(img_path)
    print("감정 예측 중...")
    prediction = model.predict(test_image)
    return prediction

# 모델 경로와 이미지 경로
model_path = '/content/drive/MyDrive/MobileNetV1-6.h5'  # 학습된 모델 경로
image_path = '/content/drive/MyDrive/testimg/5.jpg'  # 테스트할 이미지 경로

# 모델 로드
model = load_model(model_path)

# 예측 결과 얻기
predictions = predict_emotion(model, image_path)
emotion_index = np.argmax(predictions)
emotion_labels = ['화남', '행복', '슬픔', '충격']  # 실제 감정 레이블로 변경

print("예측된 감정:", emotion_labels[emotion_index])
print("각 감정의 비율:")
for i, label in enumerate(emotion_labels):
    print(f"{label}: {predictions[0][i] * 100:.2f}%")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# 모델 파일 경로
model_path = '/content/drive/MyDrive/MobileNetV1_01.h5'

# 모델 로드
model = load_model(model_path)

# 테스트 데이터 디렉토리 경로
test_data_dir = '/content/drive/MyDrive/Data/img/test'

# 테스트 데이터를 위한 ImageDataGenerator 생성
val_test_datagen = ImageDataGenerator(rescale=1./255)

# 테스트 데이터 생성기 생성
test_generator = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# 테스트 데이터 전체 검증
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# 예측 값 얻기
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# 각 클래스별 정확도 계산 및 잘못 예측한 경우 정리
misclassified_summary = []

for i, label in enumerate(class_labels):
    indices = np.where(true_classes == i)[0]
    class_accuracy = accuracy_score(true_classes[indices], predicted_classes[indices])
    print(f"Accuracy for class {label}: {class_accuracy * 100:.2f}%")

    # 잘못 예측한 경우 저장
    misclassified_indices = indices[true_classes[indices] != predicted_classes[indices]]
    for index in misclassified_indices:
        true_label = class_labels[true_classes[index]]
        predicted_label = class_labels[predicted_classes[index]]
        misclassified_summary.append((true_label, predicted_label))

# DataFrame으로 정리
misclassified_df = pd.DataFrame(misclassified_summary, columns=["True Label", "Predicted Label"])

# 잘못 예측한 경우를 집계
misclassified_counts = misclassified_df.groupby(['True Label', 'Predicted Label']).size().reset_index(name='Count')

# 정리된 결과 출력
print("\nMisclassified Cases Summary:")
print(misclassified_counts)


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

val_test_datagen = ImageDataGenerator(rescale=1./255)

# 모델 파일 경로
model_path = '/content/drive/MyDrive/MobileNetV1_04.h5'

# h5 모델 파일 로드
model = load_model(model_path)

# 테스트 데이터 전체 검증
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# 모델 파일 경로
model_path = '/content/drive/MyDrive/MobileNetV1_04.h5'

# 모델 로드
model = load_model(model_path)

# 테스트 데이터 디렉토리 경로
test_data_dir = '/content/drive/MyDrive/Data/img/test'

# 테스트 데이터를 위한 ImageDataGenerator 생성
val_test_datagen = ImageDataGenerator(rescale=1./255)

# 테스트 데이터 생성기 생성
test_generator = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# 예측 값 얻기
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# 각 클래스별 정확도 계산 및 잘못 예측한 경우 정리
misclassified_summary = []

for i, label in enumerate(class_labels):
    indices = np.where(true_classes == i)[0]
    class_accuracy = accuracy_score(true_classes[indices], predicted_classes[indices])
    print(f"Accuracy for class {label}: {class_accuracy * 100:.2f}%")

    # 잘못 예측한 경우 저장
    misclassified_indices = indices[true_classes[indices] != predicted_classes[indices]]
    for index in misclassified_indices:
        true_label = class_labels[true_classes[index]]
        predicted_label = class_labels[predicted_classes[index]]
        misclassified_summary.append((true_label, predicted_label))

# DataFrame으로 정리
misclassified_df = pd.DataFrame(misclassified_summary, columns=["True Label", "Predicted Label"])

# 잘못 예측한 경우를 집계
misclassified_counts = misclassified_df.groupby(['True Label', 'Predicted Label']).size().reset_index(name='Count')

# 정리된 결과 출력
print("\nMisclassified Cases Summary:")
print(misclassified_counts)


Found 1137 images belonging to 4 classes.
36/36 [==============================] - 77s 2s/step
Accuracy for class anger: 62.86%
Accuracy for class happy: 87.92%
Accuracy for class panic: 81.45%
Accuracy for class sadness: 66.55%

Misclassified Cases Summary:
   True Label Predicted Label  Count
0       anger           happy     14
1       anger           panic     51
2       anger         sadness     39
3       happy           anger     22
4       happy           panic      2
5       happy         sadness     12
6       panic           anger     42
7       panic           happy      6
8       panic         sadness      3
9     sadness           anger     61
10    sadness           happy     11
11    sadness           panic     23
